# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Neumann  ->  J. Neumann  |  ['J. Neumann']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']


Arxiv has 74 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2309.10038


extracting tarball to tmp_2309.10038...

 done.


Found 102 bibliographic references in tmp_2309.10038/Le_Conte_main.bbl.
syntax error in line 690: '=' expected
Retrieving document from  https://arxiv.org/e-print/2309.10316


...

 done.
Retrieving document from  https://arxiv.org/e-print/2309.10410


extracting tarball to tmp_2309.10410...

 done.


H. Beuther  ->  H. Beuther  |  ['H. Beuther']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']


Found 66 bibliographic references in tmp_2309.10410/IRAS23385_JOYS_paper2_draft_v9.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.10038-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.10038) | **A JWST investigation into the bar fraction at redshifts 1 < z < 3**  |
|| Z. A. L. Conte, et al. -- incl., <mark>J. Neumann</mark> |
|*Appeared on*| *2023-09-20*|
|*Comments*| *Submitted to MNRAS. 15 pages, 10 figures. Figure 7 summarises the main results*|
|**Abstract**| The presence of a stellar bar in a disc galaxy indicates that the galaxy hosts a dynamically settled disc and that bar-driven processes are taking place in shaping the evolution of the galaxy. Studying the cosmic evolution of the bar fraction in disc galaxies is therefore essential to understand galaxy evolution in general. Previous studies have found, using the Hubble Space Telescope (HST), that the bar fraction significantly declines from the local Universe to redshifts near one. Using the first four pointings from the James Webb Space Telescope (JWST) Cosmic Evolution Early Release Science Survey (CEERS) and the initial public observations for the Public Release Imaging for Extragalactic Research (PRIMER), we extend the studies on the bar fraction in disc galaxies to redshifts $1 \leq z \leq 3$, i.e., for the first time beyond redshift two. We only use galaxies that are also present in the Cosmic Assembly Near-IR Deep Extragalactic Legacy Survey (CANDELS) on the Extended Groth Strip (EGS) and Ultra Deep Survey (UDS) HST observations. An optimised sample of 768 close-to-face-on galaxies is visually classified to find the fraction of bars in disc galaxies in two redshift bins: $1 \leq z \leq 2$ and $2 < z \leq 3$. The bar fraction decreases from $\sim 18.9^{+ 9.7}_{- 9.4}$ per cent to $\sim 6.6^{+ 7.1}_{- 5.9}$ per cent (from the lower to the higher redshift bin), but is $\sim 3 - 4$ times greater than the bar fraction found in previous studies using bluer HST filters. Our results show that bar-driven evolution commences at early cosmic times and that dynamically settled discs are already present at a lookback time of $\sim 11$ Gyrs. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.10410-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.10410) | **JOYS: Disentangling the warm and cold material in the high-mass IRAS  23385+6053 cluster**  |
|| C. Gieser, et al. -- incl., <mark>H. Beuther</mark>, <mark>G. Perotti</mark> |
|*Appeared on*| *2023-09-20*|
|*Comments*| *15 pages, 7 figures, accepted for publication in A&A*|
|**Abstract**| (abridged) We study and compare the warm (>100 K) and cold (<100 K) material toward the high-mass star-forming region IRAS 23385+6053 (IRAS 23385 hereafter) combining high angular resolution observations in the mid-infrared (MIR) with the JWST Observations of Young protoStars (JOYS) project and with the NOEMA at mm wavelengths at angular resolutions of 0.2"-1". The spatial morphology of atomic and molecular species is investigated by line integrated intensity maps. The temperature and column density of different gas components is estimated using H2 transitions (warm and hot component) and a series of CH3CN transitions as well as 3 mm continuum emission (cold component). Toward the central dense core in IRAS 23385 the material consists of relatively cold gas and dust (~50 K), while multiple outflows create heated and/or shocked H2 and show enhanced temperatures (~400 K) along the outflow structures. An energetic outflow with enhanced emission knots of [Fe II] and [Ni II] hints at J-type shocks, while two other outflows have enhanced emission of only H2 and [S I] caused by C-type shocks. The latter two outflows are also more prominent in molecular line emission at mm wavelengths (e.g., SiO, SO, H2CO, and CH3OH). Even higher angular resolution data are needed to unambiguously identify the outflow driving sources given the clustered nature of IRAS 23385. While most of the forbidden fine structure transitions are blueshifted, [Ne II] and [Ne III] peak at the source velocity toward the MIR source A/mmA2 suggesting that the emission is originating from closer to the protostar. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.10316-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.10316) | **Constraining Thermal Emission of Pluto's Haze From Infrared Rotational  Lightcurves**  |
|| L. Wan, <mark>X. Zhang</mark>, J. D. Hofgartner |
|*Appeared on*| *2023-09-20*|
|*Comments*| *Accepted by ApJ*|
|**Abstract**| The rotational lightcurves of the Pluto-Charon system were previously believed to be solely attributed to their surfaces. However, a proposed scenario of haze cooling \citep{2017Natur.551..352Z} suggests that the atmospheric haze of Pluto could significantly contribute to mid-infrared emission, which calls for a revisit of previous analyses. In this study, we employ a Bayesian retrieval approach to constrain the haze emission from the rotational lightcurves of the Pluto-Charon system. The lightcurves were observed by the Spitzer and Herschel telescopes at 24 and 70 $\mu$m, and were combined with the latest surface albedo maps of Pluto and Charon from the New Horizons spacecraft. Our results show that including the haze emission is consistent with all current observations, with the best-fit haze flux around 1.63 mJy. This is in agreement with the composition of Titan-like tholins. However, the ``surface only" scenario, which excludes the haze contribution, can still explain the observations. We conclude that the current data at 24 $\mu$m cannot constrain Pluto's haze emission due to the degeneracy with Charon's surface emission. Regardless, some surface properties of Pluto are well constrained by the shape of the lightcurves, with a thermal inertia of approximately 8--10 MKS and a relatively low CH$_4$ emissivity of 0.3--0.5. We suggest that observations by the JWST telescope at 18 $\mu$m, which can resolve Pluto from Charon, could directly probe the haze emission of Pluto due to the low surface emission at that wavelength. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2309.10038.md
    + _build/html/tmp_2309.10038/./Figures/bar_frac.png
    + _build/html/tmp_2309.10038/./Figures/strongly_bar.png
    + _build/html/tmp_2309.10038/./Figures/weakly_bar.png
exported in  _build/html/2309.10410.md
    + _build/html/tmp_2309.10410/./continuum_IRAS23385_JWST_NOEMA.png
    + _build/html/tmp_2309.10410/./moment0_IRAS23385_H2_5.51mum_ch1_short.png
    + _build/html/tmp_2309.10410/./moment0_IRAS23385_H2_17.03mum_ch3_long.png
    + _build/html/tmp_2309.10410/./moment0_IRAS23385_SI_25.25mum_ch4_long.png
    + _build/html/tmp_2309.10410/./moment0_IRAS23385_FeII_17.94mum_ch3_long.png
    + _build/html/tmp_2309.10410/./moment0_IRAS23385_NiII_6.64mum_ch1_long.png
    + _build/html/tmp_2309.10410/./moment0_IRAS23385_NeII_12.81mum_ch3_short.png
    + _build/html/tmp_2309.10410/./moment0_IRAS23385_NeIII_15.56mum_ch3_long.png
    + _build/html/tmp_2309.10410/./moment0_IRAS23385_ArII_6.99mum_ch1_long.png
    + _build/html/tmp_2309.10410/./continuum

## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# A JWST investigation into the bar fraction at redshifts $1 \leq z \leq 3$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.10038-b31b1b.svg)](https://arxiv.org/abs/2309.10038)<mark>Appeared on: 2023-09-20</mark> -  _Submitted to MNRAS. 15 pages, 10 figures. Figure 7 summarises the main results_

</div>
<div id="authors">

Z. A. L. Conte, et al. -- incl., <mark>J. Neumann</mark>

</div>
<div id="abstract">

**Abstract:** The presence of a stellar bar in a disc galaxy indicates that the galaxy hosts a dynamically settled disc and that bar-driven processes are taking place in shaping the evolution of the galaxy. Studying the cosmic evolution of the bar fraction in disc galaxies is therefore essential to understand galaxy evolution in general. Previous studies have found, using the Hubble Space Telescope (HST), that the bar fraction significantly declines from the local Universe to redshifts near one. Using the first four pointings from the James Webb Space Telescope (JWST) Cosmic Evolution Early Release Science Survey (CEERS) and the initial public observations for the Public Release Imaging for Extragalactic Research (PRIMER), we extend the studies on the bar fraction in disc galaxies to redshifts $1 \leq z \leq 3$ , i.e., for the first time beyond redshift two. We only use galaxies that are also present in the Cosmic Assembly Near-IR Deep Extragalactic Legacy Survey (CANDELS) on the Extended Groth Strip (EGS) and Ultra Deep Survey (UDS) HST observations. An optimised sample of 768 close-to-face-on galaxies is visually classified to find the fraction of bars in disc galaxies in two redshift bins: $1 \leq z \leq 2$ and $2 < z \leq 3$ . The bar fraction decreases from $\sim 18.9^{+ 9.7}_{- 9.4}$ per cent to $\sim 6.6^{+ 7.1}_{- 5.9}$ per cent (from the lower to the higher redshift bin), but is $\sim 3 - 4$ times greater than the bar fraction found in previous studies using bluer HST filters. Our results show that bar-driven evolution commences at early cosmic times and that dynamically settled discs are already present at a lookback time of $\sim 11$ Gyrs.

</div>

<div id="div_fig1">

<img src="tmp_2309.10038/./Figures/bar_frac.png" alt="Fig10" width="100%"/>

**Figure 10. -** Evolution of the fraction of stellar bars in disc galaxies with redshift in the context of other bar assessment work using HST. The fractions of barred disc galaxies found in JWST NIRCam images are shown as green squares, and the fractions of barred disc galaxies found in this study in HST WFC3 images are shown as purple squares. The bar fraction was found for two redshift bins, $1 \leq z \leq 2$ and $2 < z \leq 3$, where the marker indicates the median redshift of the barred galaxies. All bar fraction errors indicate the sum in quadrature of the systematic and 1$\sigma$ Bayesian binomial confidence interval \citep[][statistical error only in dark colours]{Cameron_2011}  . A dashed line indicates the redshift range of barred galaxies. A thick solid line indicates the redshift range of the quartiles 25\%-75\% of the distribution of barred galaxies. At low redshifts, \citet[][down-pointing triangle]{Vaucouleurs_1991}   and \citet[][circle]{Masters_2011}   found strong bars in a third of disc galaxies, while \citet[][cross]{Eskridge_2000}   found strong and weak bars in over two-thirds of disc galaxies. \citet[][left-pointing triangles]{Simmons_2014}  , \citet[][diamonds]{Sheth_2008}   and \citet[][up-pointing triangles]{Melvin_2014}   found a decreasing trend of the bar fraction for higher redshifts. \citet[][right-pointing triangles]{Jogee_2004}   found a minimal decline in the bar fraction at higher redshifts. The six barred galaxies found in \citet[][pluses]{Guo_2023}   are shown in yellow at the bottom of the plot. Finally, the bar fractions, as found in the Auriga cosmological simulations in \citet[][exes]{Fragkoudi_2020}   are shown in black. (*Fig: bar_frac*)

</div>
<div id="div_fig2">

<img src="tmp_2309.10038/./Figures/strongly_bar.png" alt="Fig2" width="100%"/>

**Figure 2. -** Rest-frame NIR logarithmic images of strongly barred galaxies using the JWST NIRCam F444W filter between the redshifts $1 \leq z \leq 3$. The redshift of the galaxy is noted in the upper left corner of each image. A 5 kpc scale is given in the upper right corner of each image \citep[calculated using][]{Wright_2006}  . (*Fig: strong*)

</div>
<div id="div_fig3">

<img src="tmp_2309.10038/./Figures/weakly_bar.png" alt="Fig3" width="100%"/>

**Figure 3. -** Rest-frame NIR logarithmic images of weakly barred galaxies using the JWST NIRCam F444W filter between the redshifts $1 \leq z \leq 3$. The redshift of the galaxy is noted in the upper left corner of each image. A 5 kpc scale is given in the upper right corner of each image \citep[calculated using][]{Wright_2006}  . (*Fig: weak*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.10038"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# JOYS: Disentangling the warm and cold material in the high-mass IRAS 23385+6053 cluster

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.10410-b31b1b.svg)](https://arxiv.org/abs/2309.10410)<mark>Appeared on: 2023-09-20</mark> -  _15 pages, 7 figures, accepted for publication in A&A_

</div>
<div id="authors">

C. Gieser, et al. -- incl., <mark>H. Beuther</mark>, <mark>G. Perotti</mark>

</div>
<div id="abstract">

**Abstract:** High-mass star formation occurs in a clustered mode where fragmentation is observed from an early stage onward. Young protostars can now be studied in great detail with the recently launched _James Webb Space Telescope_ (JWST). We study and compare the warm ( $>$ 100 K) and cold ( $<$ 100 K) material toward the high-mass star-forming region IRAS 23385+6053 (IRAS 23385 hereafter) combining high angular resolution observations in the mid-infrared (MIR) with the JWST Observations of Young protoStars (JOYS) project and with the NOrthern Extended Millimeter Array (NOEMA) at mm wavelengths at angular resolutions of $\approx$ 0 $\farcs$ 2-1 $\farcs$ 0. The spatial morphology of atomic and molecular species is investigated by line integrated intensity maps. The temperature and column density of different gas components is estimated using H $_{2}$ transitions (warm and hot component) and a series of CH $_{3}$ CN transitions as well as 3 mm continuum emission (cold component). Toward the central dense core in IRAS 23385 the material consists of relatively cold gas and dust ( $\approx$ 50 K), while multiple outflows create heated and/or shocked H $_{2}$ and show enhanced temperatures ( $\approx$ 400 K) along the outflow structures. An energetic outflow with enhanced emission knots of $[$ Fe ${\sc ii}$ $]$ and $[$ Ni ${\sc ii}$ $]$ hints at $J$ -type shocks, while two other outflows have enhanced emission of only H $_{2}$ and $[$ S ${\sc i}$ $]$ caused by $C$ -type shocks. The latter two outflows are also more prominent in molecular line emission at mm wavelengths (e.g., SiO, SO, H $_{2}$ CO, and CH $_{3}$ OH). Even higher angular resolution data are needed to unambiguously identify the outflow driving sources given the clustered nature of IRAS 23385. While most of the forbidden fine structure transitions are blueshifted, $[$ Ne ${\sc ii}$ $]$ and $[$ Ne ${\sc iii}$ $]$ peak at the source velocity toward the MIR source A/mmA2 suggesting that the emission is originating from closer to the protostar. The warm and cold gas traced by MIR and mm observations, respectively, are strongly linked in IRAS 23385. The outflows traced by MIR H $_{2}$ lines have molecular counterparts in the mm regime. Despite the presence of multiple powerful outflows that cause dense and hot shocks, a cold dense envelope still allows star formation to further proceed. To study and fully understand the spatially resolved MIR-properties, a representative sample of low- and high-mass protostars has to be probed by JWST.

</div>

<div id="div_fig1">

<img src="tmp_2309.10410/./continuum_IRAS23385_JWST_NOEMA.png" alt="Fig5.1" width="50%"/><img src="tmp_2309.10410/./moment0_IRAS23385_H2_5.51mum_ch1_short.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** MIR Continuum (_left_) and H$_{2}$ 0-0 S(7) line emission (_right_) for IRAS 23385. In the left panel, the JWST/MIRI 5.2 $\upmu$m continuum with emission $> 5\sigma_{\mathrm{cont,5}\upmu\mathrm{m}}$ is presented in color. Grey contours show the NOEMA 1 mm continuum with contour levels at 5, 10, 20, 40, 80$\times \sigma_\mathrm{cont,1mm}$. The dash-dotted white circles show the aperture in which the 5.2 $\upmu$m flux density $F_{5.2\upmu\mathrm{m}}$ was derived (Table \ref{tab:sources}). In the bottom left corner, the JWST/MIRI 5.2 $\upmu$m and NOEMA 1 mm angular resolution is indicated in black and grey, respectively. The mm (mmA1 and mmB, marked by green squares) and MIR (A/mmA2 and B) continuum sources are labeled in green. In the right panel, the line integrated intensity of the H$_{2}$ 0-0 S(7) line with $S$/$N > 5$ is presented in color. The JWST/MIRI 5.2 $\upmu$m continuum as shown in the left panel is presented as blue contours with contour levels at 5, 10, 15, 20, 25$\times \sigma_{\mathrm{cont,5}\upmu\mathrm{m}}$. The angular resolution of the 5.2 $\upmu$m continuum and H$_{2}$ 0-0 S(7) line data is indicated in the bottom left and right, respectively. Several shock spots evident in the MIRI MRS line emission are marked by green crosses and labeled in green (Sect. \ref{sec:line}). The red and blue arrows indicate three bipolar outflows, labeled I, II, and III \citep[as presented in][]{Beuther2023}. In both panels, the black bar indicates a spatial scale of 10 000 au at the assumed source distance of 4.9 kpc. (*fig:continuum*)

</div>
<div id="div_fig2">

<img src="tmp_2309.10410/./moment0_IRAS23385_H2_17.03mum_ch3_long.png" alt="Fig6.1" width="14%"/><img src="tmp_2309.10410/./moment0_IRAS23385_SI_25.25mum_ch4_long.png" alt="Fig6.2" width="14%"/><img src="tmp_2309.10410/./moment0_IRAS23385_FeII_17.94mum_ch3_long.png" alt="Fig6.3" width="14%"/><img src="tmp_2309.10410/./moment0_IRAS23385_NiII_6.64mum_ch1_long.png" alt="Fig6.4" width="14%"/><img src="tmp_2309.10410/./moment0_IRAS23385_NeII_12.81mum_ch3_short.png" alt="Fig6.5" width="14%"/><img src="tmp_2309.10410/./moment0_IRAS23385_NeIII_15.56mum_ch3_long.png" alt="Fig6.6" width="14%"/><img src="tmp_2309.10410/./moment0_IRAS23385_ArII_6.99mum_ch1_long.png" alt="Fig6.7" width="14%"/>

**Figure 6. -** Integrated intensity maps of atomic and molecular lines detected with JWST/MIRI MRS. In color, the line integrated intensity is shown in a log-scale. The JWST/MIRI 5.2 $\upmu$m continuum is presented as blue contours with contour levels at 5, 10, 15, 20, 25$\times \sigma_{\mathrm{cont,5}\upmu\mathrm{m}}$. The two mm sources are indicated by green squares and several shock positions are highlighted by green crosses. In the bottom left and right corners, the angular resolution of the JWST/MIRI continuum and line data, respectively, is shown. In the top left panel, the bipolar outflows are indicated by red and blue dashed arrows. The shock locations (Sect. \ref{sec:line}) and continuum sources are labeled in the top right and center left panel, respectively. In the $[$Ne {\sc ii}$]$ and $[$Ne {\sc iii}$]$ panels, the dash-dotted grey circles show the aperture (0$\farcs$9) in which the flux density was derived (Sect. \ref{sec:discussion}). (*fig:line*)

</div>
<div id="div_fig3">

<img src="tmp_2309.10410/./continuum_IRAS23385_overview.png" alt="Fig4" width="100%"/>

**Figure 4. -** Multi-wavelength overview of IRAS 23385. In color, the _Herschel_ 250 $\upmu$m (_left_), _Herschel_ 70 $\upmu$m (_center_), and _Spitzer_ 24 $\upmu$m (_right_) emission are shown in log-scale. The grey dashed squares show the field-of-view of the following panel. In the center and right panels, the green star, labeled as "mm", marks the 1 mm continuum peak position (Fig. \ref{fig:continuum}). The blue stars, labeled as "VLA1" and "VLA2", show the position of the two nearby UCH{\sc ii} regions  ([Molinari, et. al 2002]()) . The black rectangles in the right panel indicate the JWST/MIRI MRS field-of-view of the 4-pointing mosaic in ch1A (solid) and ch4C (dashed). (*fig:overview*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.10410"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

275  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
